Model prognozujący frekwencję na meczach domowych Jagiellonii Białystok. Projekt stworzony w celu poznania języka Python oraz zagadanień uczenia maszynowego. Tematyka związana z własnymi zainteresowaniami piłką nożną i białostockim klubem. 

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import plotly.express as px

Zebrałem dane o frekwencji na meczach domowych Jagiellonii Białystok w sezonach 2015/2016 - 2018/2019
oraz 2022/2023 - 2023/2024. Sezony 2019/2020 - 2021/2022 zostały przeze mnie pominięte z powodu okresowych ograniczeń covidowych wpływających na frekwencję i mogących nieodpowiednio wpływać na późniejsze predykcje. Dane zawierają informacje o dniu tygodnia i godzinie rozegrania meczu, temperaturze, pozycji Jagiellonii przed meczem oraz rywalu. Dane zebrano na podstawie serwisów: transfermarkt.pl, 90minut.pl oraz wikipedia.org. 



In [2]:
df = pd.read_csv('./Dane1.csv')

In [3]:
df.head(10)

,sezon,dzien_tygodnia,godzina,temperatura,pozycja_przed_meczem,rywal,frekwencja
0,2015/2016,niedziela,15:30,16,11,Nieciecza,10653
1,2015/2016,niedziela,15:30,26,7,Zagłębie Lubin,11055
2,2015/2016,niedziela,15:30,28,7,Ruch Chorzów,12758
3,2015/2016,niedziela,18:00,26,4,Legia Warszawa,22172
4,2015/2016,niedziela,18:00,15,10,Lech Poznań,19177
5,2015/2016,piątek,20:30,9,7,Lechia Gdańsk,11195
6,2015/2016,piątek,20:30,10,11,Wisła Kraków,10120
7,2015/2016,sobota,20:30,3,13,Piast Gliwice,8523
8,2015/2016,sobota,15:30,4,13,Korona Kielce,7120
9,2015/2016,piątek,20:30,7,11,Górnik Zabrze,6734


In [3]:
df.shape

(105, 7)

Sprawdzenie czy dane zawierają duplikaty

In [4]:
df.duplicated().sum()

0

Sprawdzenie czy istnieją puste wartości

In [5]:
df.isna().sum()

sezon                   0
dzien_tygodnia          0
godzina                 0
temperatura             0
pozycja_przed_meczem    0
rywal                   0
frekwencja              0
dtype: int64

Wyświetlenie podstawowych informacji o zbiorze danych

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   sezon                 105 non-null    object
 1   dzien_tygodnia        105 non-null    object
 2   godzina               105 non-null    object
 3   temperatura           105 non-null    int64 
 4   pozycja_przed_meczem  105 non-null    int64 
 5   rywal                 105 non-null    object
 6   frekwencja            105 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 5.9+ KB


Wyświetlenie statystyk opisowych danych numerycznych 

In [7]:
df.describe().round()

,temperatura,pozycja_przed_meczem,frekwencja
count,105.0,105.0,105.0
mean,12.0,5.0,11005.0
std,9.0,4.0,4257.0
min,-11.0,1.0,4111.0
25%,5.0,2.0,8123.0
50%,11.0,3.0,10006.0
75%,18.0,9.0,13240.0
max,30.0,18.0,22394.0


Obliczenie macierzy korelacji

In [8]:
macierz_korelacji = df.corr()

In [9]:
print(macierz_korelacji)

                      temperatura  pozycja_przed_meczem  frekwencja
temperatura              1.000000              0.056218    0.511215
pozycja_przed_meczem     0.056218              1.000000   -0.332025
frekwencja               0.511215             -0.332025    1.000000


Postanowiłem zakodować kolumnę 'rywal' na podstawie średniej frekwencji na meczach z danym rywalem. 

In [10]:
srednia_frekwencja = df.groupby('rywal')['frekwencja'].mean().sort_values().reset_index().round()

In [11]:
srednia_frekwencja

,rywal,frekwencja
0,Z. Sosnowiec,6541.0
1,Górnik Łęczna,7390.0
2,Korona Kielce,7488.0
3,Stal Mielec,7720.0
4,Zagłębie Lubin,8218.0
5,Piast Gliwice,8260.0
6,Raków,8328.0
7,P. Niepołomice,8478.0
8,Warta Poznań,8481.0
9,Podbeskidzie,8792.0


In [12]:
df['rywal'].unique()

array(['Nieciecza', 'Zagłębie Lubin', 'Ruch Chorzów', 'Legia Warszawa',
       'Lech Poznań', 'Lechia Gdańsk', 'Wisła Kraków', 'Piast Gliwice',
       'Korona Kielce', 'Górnik Zabrze', 'Cracovia', 'Śląsk Wrocław',
       'Pogoń Szczecin', 'Górnik Łęczna', 'Podbeskidzie', 'Arka Gdynia',
       'Piast Gliwice\xa0', 'Wisła Płock', 'Sandecja', '\tWisła Kraków',
       'Miedź Legnica', 'Z. Sosnowiec', 'Widzew Łódź', 'Radomiak',
       'Stal Mielec', 'Legia Warszawa\xa0', 'Raków', 'Warta Poznań',
       'P. Niepołomice', 'ŁKS Łódź'], dtype=object)

W kolumnie 'rywal' istniały białe znaki. Usunąłem je i ponownie obliczyłem średnią frekwencję. 

In [13]:
df['rywal'] = df['rywal'].str.strip()

In [14]:
df['rywal'].unique()

array(['Nieciecza', 'Zagłębie Lubin', 'Ruch Chorzów', 'Legia Warszawa',
       'Lech Poznań', 'Lechia Gdańsk', 'Wisła Kraków', 'Piast Gliwice',
       'Korona Kielce', 'Górnik Zabrze', 'Cracovia', 'Śląsk Wrocław',
       'Pogoń Szczecin', 'Górnik Łęczna', 'Podbeskidzie', 'Arka Gdynia',
       'Wisła Płock', 'Sandecja', 'Miedź Legnica', 'Z. Sosnowiec',
       'Widzew Łódź', 'Radomiak', 'Stal Mielec', 'Raków', 'Warta Poznań',
       'P. Niepołomice', 'ŁKS Łódź'], dtype=object)

In [15]:
srednia_frekwencja = df.groupby('rywal')['frekwencja'].mean().sort_values().reset_index().round()

In [16]:
srednia_frekwencja

,rywal,frekwencja
0,Z. Sosnowiec,6541.0
1,Górnik Łęczna,7390.0
2,Korona Kielce,7488.0
3,Stal Mielec,7720.0
4,Zagłębie Lubin,8218.0
5,Raków,8328.0
6,P. Niepołomice,8478.0
7,Warta Poznań,8481.0
8,Podbeskidzie,8792.0
9,Górnik Zabrze,8852.0


Zakodowałem kolumnę 'rywal'.

In [17]:
rywale = {rywal: i for i, rywal in enumerate(srednia_frekwencja['rywal'])}

In [18]:
rywale

{'Z. Sosnowiec': 0,
 'Górnik Łęczna': 1,
 'Korona Kielce': 2,
 'Stal Mielec': 3,
 'Zagłębie Lubin': 4,
 'Raków': 5,
 'P. Niepołomice': 6,
 'Warta Poznań': 7,
 'Podbeskidzie': 8,
 'Górnik Zabrze': 9,
 'Piast Gliwice': 10,
 'Miedź Legnica': 11,
 'Pogoń Szczecin': 12,
 'Śląsk Wrocław': 13,
 'Arka Gdynia': 14,
 'Nieciecza': 15,
 'Radomiak': 16,
 'Lechia Gdańsk': 17,
 'Sandecja': 18,
 'Widzew Łódź': 19,
 'Cracovia': 20,
 'Wisła Płock': 21,
 'Ruch Chorzów': 22,
 'Wisła Kraków': 23,
 'Lech Poznań': 24,
 'ŁKS Łódź': 25,
 'Legia Warszawa': 26}

In [19]:
df['kod_rywala'] = df['rywal'].map(rywale)

In [20]:
df

,sezon,dzien_tygodnia,godzina,temperatura,pozycja_przed_meczem,rywal,frekwencja,kod_rywala
0,2015/2016,niedziela,15:30,16,11,Nieciecza,10653,15
1,2015/2016,niedziela,15:30,26,7,Zagłębie Lubin,11055,4
2,2015/2016,niedziela,15:30,28,7,Ruch Chorzów,12758,22
3,2015/2016,niedziela,18:00,26,4,Legia Warszawa,22172,26
4,2015/2016,niedziela,18:00,15,10,Lech Poznań,19177,24
...,...,...,...,...,...,...,...,...
100,2023/2024,sobota,15:00,11,1,Ruch Chorzów,11098,22
101,2023/2024,piątek,20:30,-2,3,Śląsk Wrocław,13840,13
102,2023/2024,sobota,15:00,22,1,ŁKS Łódź,15007,25
103,2023/2024,niedziela,15:00,16,1,Cracovia,18184,20


Obliczyłem macierz korelacji

In [21]:
macierz_korelacji = df.corr()

In [22]:
print(macierz_korelacji)

                      temperatura  pozycja_przed_meczem  frekwencja  \
temperatura              1.000000              0.056218    0.511215   
pozycja_przed_meczem     0.056218              1.000000   -0.332025   
frekwencja               0.511215             -0.332025    1.000000   
kod_rywala               0.197674             -0.244795    0.624442   

                      kod_rywala  
temperatura             0.197674  
pozycja_przed_meczem   -0.244795  
frekwencja              0.624442  
kod_rywala              1.000000  


Wizualizacja relacji między temperaturą a frekwencją na meczach

In [24]:
sns.set_style("whitegrid")


fig = px.scatter(df, x='temperatura', y='frekwencja', color='sezon', hover_data=['pozycja_przed_meczem', 'rywal'],
                 title='Temperatura a frekwencja na meczach', 
                 labels={'temperatura': 'Temperatura (°C)', 'frekwencja': 'Frekwencja', 'sezon': 'Sezon'},
                 template='plotly')  


fig.update_xaxes(title_text='Temperatura (°C)')
fig.update_yaxes(title_text='Frekwencja')

fig.show()



Kolumny dzien_tygodnia oraz godzina postanowiłem zakodować metodą one-hot.

In [25]:
kody_dni = pd.get_dummies(df['dzien_tygodnia'], prefix='dzien')

In [26]:
kody_godzin = pd.get_dummies(df['godzina'], prefix='godzina')

In [28]:
df1 = pd.concat([df, kody_dni, kody_godzin], axis=1)

In [29]:
df1

,sezon,dzien_tygodnia,godzina,temperatura,pozycja_przed_meczem,rywal,frekwencja,kod_rywala,dzien_niedziela,dzien_piątek,...,dzien_wtorek,dzien_środa,godzina_12:30,godzina_15:00,godzina_15:30,godzina_17:30,godzina_18:00,godzina_19:00,godzina_20:00,godzina_20:30
0,2015/2016,niedziela,15:30,16,11,Nieciecza,10653,15,1,0,...,0,0,0,0,1,0,0,0,0,0
1,2015/2016,niedziela,15:30,26,7,Zagłębie Lubin,11055,4,1,0,...,0,0,0,0,1,0,0,0,0,0
2,2015/2016,niedziela,15:30,28,7,Ruch Chorzów,12758,22,1,0,...,0,0,0,0,1,0,0,0,0,0
3,2015/2016,niedziela,18:00,26,4,Legia Warszawa,22172,26,1,0,...,0,0,0,0,0,0,1,0,0,0
4,2015/2016,niedziela,18:00,15,10,Lech Poznań,19177,24,1,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,2023/2024,sobota,15:00,11,1,Ruch Chorzów,11098,22,0,0,...,0,0,0,1,0,0,0,0,0,0
101,2023/2024,piątek,20:30,-2,3,Śląsk Wrocław,13840,13,0,1,...,0,0,0,0,0,0,0,0,0,1
102,2023/2024,sobota,15:00,22,1,ŁKS Łódź,15007,25,0,0,...,0,0,0,1,0,0,0,0,0,0
103,2023/2024,niedziela,15:00,16,1,Cracovia,18184,20,1,0,...,0,0,0,1,0,0,0,0,0,0


Obliczenie macierzy korelacji

In [30]:
macierz_korelacji = df1.corr()

In [32]:
print(macierz_korelacji)

                      temperatura  pozycja_przed_meczem  frekwencja  \
temperatura              1.000000              0.056218    0.511215   
pozycja_przed_meczem     0.056218              1.000000   -0.332025   
frekwencja               0.511215             -0.332025    1.000000   
kod_rywala               0.197674             -0.244795    0.624442   
dzien_niedziela          0.404575             -0.168470    0.451255   
dzien_piątek            -0.251663             -0.033395   -0.170172   
dzien_poniedziałek      -0.018114              0.070279   -0.176588   
dzien_sobota            -0.144022              0.151654   -0.164216   
dzien_wtorek             0.002294              0.065951   -0.129066   
dzien_środa             -0.072988             -0.012562   -0.025482   
godzina_12:30            0.107567             -0.053039    0.025139   
godzina_15:00            0.113335              0.144010    0.042013   
godzina_15:30            0.113976              0.072898   -0.103339   
godzin

Podział na zmienne objaśniające i zmienną objaśnianą 

In [33]:
X = df1[['temperatura', 'pozycja_przed_meczem', 'kod_rywala', 'dzien_niedziela']]
y = df1['frekwencja']


Import oraz inicjalizacja modeli regresji liniowej, drzewa decyzyjnego, lasu losowego oraz SVR

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

regresja_liniowa = LinearRegression()
drzewo_decyzyjne = DecisionTreeRegressor(random_state=2020)
las_losowy = RandomForestRegressor(random_state=2020)
model_svr = SVR()

Wykonanie predykcji z wykorzystaniem walidacji krzyżowej oraz obliczenie wartości błędów średniokwadratowych

In [35]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error


y_pred_regresja_liniowa = cross_val_predict(regresja_liniowa, X, y, cv=5)
y_pred_drzewo_decyzyjne = cross_val_predict(drzewo_decyzyjne, X, y, cv=5)
y_pred_las_losowy = cross_val_predict(las_losowy, X, y, cv=5)
y_pred_model_svr = cross_val_predict(model_svr, X, y, cv=5)


mse_regresja_liniowa = mean_squared_error(y, y_pred_regresja_liniowa)
mse_drzewo_decyzyjne = mean_squared_error(y, y_pred_drzewo_decyzyjne)
mse_las_losowy = mean_squared_error(y, y_pred_las_losowy)
mse_model_svr = mean_squared_error(y, y_pred_model_svr)

print("Błąd średniokwadratowy dla regresji liniowej:", mse_regresja_liniowa)
print("Błąd średniokwadratowy dla drzewa decyzyjnego:", mse_drzewo_decyzyjne)
print("Błąd średniokwadratowy dla lasu losowego:", mse_las_losowy)
print("Błąd średniokwadratowy dla SVR:", mse_model_svr)

Błąd średniokwadratowy dla regresji liniowej: 7226688.944444552
Błąd średniokwadratowy dla drzewa decyzyjnego: 13379642.0
Błąd średniokwadratowy dla lasu losowego: 7001802.286202857
Błąd średniokwadratowy dla SVR: 20353878.36920744


Obliczenie współczynnika R2 z wykorzystaniem walidacji krzyżowej

In [37]:
from sklearn.model_selection import cross_val_score

# Obliczenie R2 dla każdego modelu za pomocą walidacji krzyżowej
r2_regresja_liniowa = cross_val_score(regresja_liniowa, X, y, cv=5, scoring='r2').mean()
r2_drzewo_decyzyjne = cross_val_score(drzewo_decyzyjne, X, y, cv=5, scoring='r2').mean()
r2_las_losowy = cross_val_score(las_losowy, X, y, cv=5, scoring='r2').mean()
r2_model_svr = cross_val_score(model_svr, X, y, cv=5, scoring='r2').mean()

print("R2 dla regresji liniowej:", r2_regresja_liniowa)
print("R2 dla drzewa decyzyjnego:", r2_drzewo_decyzyjne)
print("R2 dla lasu losowego:", r2_las_losowy)
print("R2 dla SVR:", r2_model_svr)

R2 dla regresji liniowej: 0.5507090889895392
R2 dla drzewa decyzyjnego: 0.20700358741513508
R2 dla lasu losowego: 0.5624161087629399
R2 dla SVR: -0.23670360160390708


Obliczenie kolejnych metryk

In [38]:
mae_regresja_liniowa = -cross_val_score(regresja_liniowa, X, y, cv=5, scoring='neg_mean_absolute_error').mean()
mae_drzewo_decyzyjne = -cross_val_score(drzewo_decyzyjne, X, y, cv=5, scoring='neg_mean_absolute_error').mean()
mae_las_losowy = -cross_val_score(las_losowy, X, y, cv=5, scoring='neg_mean_absolute_error').mean()
mae_model_svr = -cross_val_score(model_svr, X, y, cv=5, scoring='neg_mean_absolute_error').mean()

print("Mean Absolute Error dla regresji liniowej:", mae_regresja_liniowa)
print("Mean Absolute Error dla drzewa decyzyjnego:", mae_drzewo_decyzyjne)
print("Mean Absolute Error dla lasu losowego:", mae_las_losowy)
print("Mean Absolute Error dla SVR:", mae_model_svr)

evs_regresja_liniowa = cross_val_score(regresja_liniowa, X, y, cv=5, scoring='explained_variance').mean()
evs_drzewo_decyzyjne = cross_val_score(drzewo_decyzyjne, X, y, cv=5, scoring='explained_variance').mean()
evs_las_losowy = cross_val_score(las_losowy, X, y, cv=5, scoring='explained_variance').mean()
evs_model_svr = cross_val_score(model_svr, X, y, cv=5, scoring='explained_variance').mean()

print("Explained Variance Score dla regresji liniowej:", evs_regresja_liniowa)
print("Explained Variance Score dla drzewa decyzyjnego:", evs_drzewo_decyzyjne)
print("Explained Variance Score dla lasu losowego:", evs_las_losowy)
print("Explained Variance Score dla SVR:", evs_model_svr)


Mean Absolute Error dla regresji liniowej: 2096.8036495386773
Mean Absolute Error dla drzewa decyzyjnego: 2729.6761904761906
Mean Absolute Error dla lasu losowego: 1941.927333333333
Mean Absolute Error dla SVR: 3466.797928656038
Explained Variance Score dla regresji liniowej: 0.627817661237746
Explained Variance Score dla drzewa decyzyjnego: 0.30218067016400896
Explained Variance Score dla lasu losowego: 0.6626189363679709
Explained Variance Score dla SVR: 0.0036830584159085245


Na podstawie wyników metryk wybrałem model lasu losowego. Dostrojenie hiperparametrów za pomocą przeszukiwania siatkowego. 

In [40]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}



grid_search = GridSearchCV(estimator=las_losowy, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)


print("Najlepsze parametry:", grid_search.best_params_)


print("Najlepszy wynik:", -grid_search.best_score_)


Najlepsze parametry: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Najlepszy wynik: 7001802.286202855


Inicjalizacja ostatecznego modelu lasu losowego z najlepszymi hiperparametrami

In [41]:
las_losowy_final = RandomForestRegressor(n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=2020)

las_losowy_final.fit(X, y)

RandomForestRegressor(random_state=2020)

Predykcja na podstawie temperatury, pozycji drużyny w tabeli przed meczem, kodu rywala oraz informacji o tym czy mecz jest w niedzielę (0 - nie, 1 - tak)

In [42]:
kody = df1[['rywal', 'kod_rywala']]

In [43]:
kody = kody.drop_duplicates()

In [45]:
kody.set_index('rywal').sort_values(by='rywal')

,kod_rywala
rywal,
Arka Gdynia,14
Cracovia,20
Górnik Zabrze,9
Górnik Łęczna,1
Korona Kielce,2
Lech Poznań,24
Lechia Gdańsk,17
Legia Warszawa,26
Miedź Legnica,11


In [59]:
las_losowy_final.predict([[17,2,24,1]])[0].round()

16826.0